In [1]:
# mhcflurry_cloud example
# Runs cross validation over a few models.
#
#
# **** BEFORE RUNNING ****
# You must start celery in an environment (e.g. venv) where mhcflurry_cloud is installed.
# To do so, run this command in a shell:
#     celery -A mhcflurry_cloud.celery.app worker --loglevel=info
#
#
import mhcflurry_cloud
import mhcflurry
import fancyimpute

/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
# Load some data.
train_data = mhcflurry.dataset.Dataset.from_csv("../test/data/bdata.2009.mhci.public.1.txt")
train_data

Dataset(n=137654, alleles=['ELA-A1', 'Gogo-B0101', 'H-2-DB', 'H-2-DD', 'H-2-KB', 'H-2-KD', 'H-2-KK', 'H-2-LD', 'HLA-A0101', 'HLA-A0201', 'HLA-A0202', 'HLA-A0203', 'HLA-A0205', 'HLA-A0206', 'HLA-A0207', 'HLA-A0210', 'HLA-A0211', 'HLA-A0212', 'HLA-A0216', 'HLA-A0219', 'HLA-A0250', 'HLA-A0301', 'HLA-A0302', 'HLA-A11', 'HLA-A1101', 'HLA-A2', 'HLA-A2301', 'HLA-A2402', 'HLA-A2403', 'HLA-A2501', 'HLA-A26', 'HLA-A2601', 'HLA-A2602', 'HLA-A2603', 'HLA-A2902', 'HLA-A3001', 'HLA-A3002', 'HLA-A3101', 'HLA-A3201', 'HLA-A3301', 'HLA-A6601', 'HLA-A6801', 'HLA-A6802', 'HLA-A6901', 'HLA-A8001', 'HLA-B0702', 'HLA-B0801', 'HLA-B0802', 'HLA-B0803', 'HLA-B1402', 'HLA-B1501', 'HLA-B1502', 'HLA-B1503', 'HLA-B1509', 'HLA-B1517', 'HLA-B1801', 'HLA-B2701', 'HLA-B2702', 'HLA-B2703', 'HLA-B2705', 'HLA-B3501', 'HLA-B3503', 'HLA-B3508', 'HLA-B3801', 'HLA-B3901', 'HLA-B4001', 'HLA-B4002', 'HLA-B4201', 'HLA-B44', 'HLA-B4402', 'HLA-B4403', 'HLA-B4501', 'HLA-B4601', 'HLA-B4801', 'HLA-B5101', 'HLA-B5301', 'HLA-B5401', '

In [ ]:
# Generate cross validation folds, with imputation.
imputer = fancyimpute.MICE(
        n_imputations=2, n_burn_in=1, n_nearest_columns=25)

folds = mhcflurry_cloud.cross_validation_folds(
    train_data,
    n_folds=3,
    imputer=imputer,
    drop_similar_peptides=True,
    alleles=["HLA-A0201", "HLA-A0202"],
)
folds

In [ ]:
# Make some models.
models = mhcflurry_cloud.models_grid(
    activation=["tanh", "relu"],
    layer_sizes=[[4]],
    embedding_output_dim=[8],
    n_training_epochs=[3])
models

In [ ]:
# Train and test models on the folds.
df = mhcflurry_cloud.train_and_test_across_models_and_folds(
    folds,
    models)
df